# ETL

In [130]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


df = pd.read_csv("C:/Users/pablo/Desktop/proyecto_final/2017PurchasePricesDec.csv")

df.isnull().sum()


df.dropna(how='any', inplace=True)

df['Size'] = df['Size'].str.lower()

df['Size'].unique()


array(['750ml', '1000ml', '1750ml', '50ml', '375ml', '100ml 4 pk',
       '50ml 5 pk', '100ml', '200ml', '50ml 4 pk', '50ml 3 pk', '300ml',
       '200ml 4 pk', 'unknown', '750ml 2 pk', '250ml 4 pk', '1500ml',
       '3000ml', '5000ml', '4000ml', '187ml 4 pk', '150ml', '187ml',
       '500ml', '720ml', '650ml', '200ml 3 pk', '187ml 3 pk', '330ml',
       '250ml', '750ml + 3/', '18000ml', '180ml', '750ml + 4/', '6000ml',
       '1000ml 2 pk', '750ml 12 p', '750ml 6 pk', '20ml 5 pk',
       '375ml 2 pk', '20000ml', '50ml 12 pk', '750ml 3 pk', '375ml 3 pk',
       '750ml + 1/', '200ml 5 pk', '162.5ml', '400ml', '1100ml', '600ml',
       '19500ml', '560ml', '3750ml', '750ml  3', '9000ml'], dtype=object)

In [131]:
df_work = df.copy()

# Normalización
s = (df_work['Size'].astype('string').str.lower().str.strip()
       .str.replace(r'\s+', ' ', regex=True)
       .str.replace(r'\bpk\.?\b', ' pk', regex=True)     # "pk." -> " pk"
       .str.replace(r'(?<=\d)\s+p\b', ' pk', regex=True) # "12 p" -> "12 pk"
)


In [132]:
pat = (
    r'^\s*'
    r'(?P<ml_val>\d+(?:\.\d+)?)\s*ml'            # 50ml | 162.5ml | 1750ml
    r'(?:\s*(?P<qty>\d{1,3})(?:\s*pk)?)?'        # "12 pk" o "ml 12" 
    r'(?:\s*\+\s*(?P<free>\d+)\s*/)?'            # "+ 3/" 
    r'\s*$'
)
m = s.str.extract(pat)


In [133]:
# Construir las DOS columnas
#  'size_ml' como string "Nml" (redondeo por si aparece 162.5ml)
ml_num = (pd.to_numeric(m['ml_val'], errors='coerce').round().astype('Int64'))
size_ml = (ml_num.astype('string') + 'ml').where(ml_num.notna(), pd.NA)

In [134]:
qty_pk = pd.to_numeric(m['qty'],  errors='coerce').astype('Int64')
free   = pd.to_numeric(m['free'], errors='coerce').astype('Int64')


In [135]:
size_qty = qty_pk.where(qty_pk.notna(), free)      # si no hay pk, usa free
size_qty = size_qty.where(size_qty.notna(), 1)     # si no hay nada, 1
size_qty = size_qty.where(ml_num.notna(), pd.NA)   # si no hay tamaño, NA

In [146]:
# 4) Asignar al DataFrame final (solo dos columnas nuevas)
df_work['size_ml']  = size_ml
df_work['size_qty'] = size_qty

df_work['size_ml'].unique(),df_work['size_qty'].unique()


df_work['size_ml'].isna().sum(),df_work['size_qty'].isna().sum()

df_work.head()



,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName,size_ml,size_qty
0,58,Gekkeikan Black & Gold Sake,12.99,750ml,750,1,9.28,8320,SHAW ROSS INT L IMP LTD,750ml,1
1,62,Herradura Silver Tequila,36.99,750ml,750,1,28.67,1128,BROWN-FORMAN CORP,750ml,1
2,63,Herradura Reposado Tequila,38.99,750ml,750,1,30.46,1128,BROWN-FORMAN CORP,750ml,1
3,72,No. 3 London Dry Gin,34.99,750ml,750,1,26.11,9165,ULTRA BEVERAGE COMPANY LLP,750ml,1
4,75,Three Olives Tomato Vodka,14.99,750ml,750,1,10.94,7245,PROXIMO SPIRITS INC.,750ml,1


In [147]:
df_work.to_csv("C:/Users/pablo/Desktop/proyecto_final/2017PurchasePricesDec_limpio.csv")